In [1]:
import nltk

In [2]:
##downloading package

nltk.download_shell()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> l

Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [ ] cess_esp............ CESS-ESP Treebank
  [ ] chat80.....

        C:\Users\singh\AppData\Roaming\nltk_data...
      Unzipping corpora\stopwords.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


In [99]:
##open dataset "Smsspamcollection"

messages=[line.rstrip() for line in open('smsspamcollection/SMSSpamCollection')]
print(len(messages))
messages[101]

5574


'ham\tOkay name ur price as long as its legal! Wen can I pick them up? Y u ave x ams xx'

In [100]:
# for mess_no,message in enumerate(messages[:20]):
#     print(message)
#     print('\n')

In [101]:
import pandas as pd

In [102]:
##open dataset in form of .csv file

messages=pd.read_csv('smsspamcollection/SMSSpamCollection',sep='\t',names=['Label','Message']) ##giving names to row
messages

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [103]:
messages.groupby('Label').describe()

Message                                                               
        count unique                                                top freq
Label                                                                       
ham      4825   4516                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

In [104]:
messages['length']=messages['Message'].apply(len)
messages.head()

,Label,Message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [98]:
import string

In [59]:
from nltk.corpus import stopwords

In [60]:
# stopwords.words('english')

# Tokenization

In [66]:
def text_cleaning(mess):
    nopunc=[char for char in mess if mess not in string.punctuation]                           ##remoing punctuation
    nopunc=''.join(nopunc)                                                                     ##join message 
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')] ##removes stopwords

In [67]:
messages.head()

,Label,Message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [68]:
messages['Message'].head(5).apply(text_cleaning)

0    [Go, jurong, point,, crazy.., Available, bugis...
1                 [Ok, lar..., Joking, wif, u, oni...]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3    [U, dun, say, early, hor..., U, c, already, sa...
4      [Nah, think, goes, usf,, lives, around, though]
Name: Message, dtype: object

In [69]:
from sklearn.feature_extraction.text import CountVectorizer

In [71]:
bow=CountVectorizer(analyzer=text_cleaning).fit(messages['Message'])

In [72]:
print(len(bow.vocabulary_))

15333


In [73]:
mess4=messages['Message'][3]

In [74]:
print(mess4)

U dun say so early hor... U c already then say...


In [75]:
b=bow.transform([mess4])
# print(b.shape)
print(b)

  (0, 5042)	2
  (0, 5760)	1
  (0, 6635)	1
  (0, 7936)	1
  (0, 7963)	1
  (0, 9331)	1
  (0, 12680)	1
  (0, 12684)	1


In [76]:
bow.get_feature_names()[6635]

'c'

In [77]:
message_bow=bow.transform(messages['Message'])
print("Shape of matrix ",message_bow.shape)

Shape of matrix  (5572, 15333)


# Term-Frequency_Inverse_Document_Frequency

In [37]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transform=TfidfTransformer().fit(message_bow)

In [78]:
message_tfidf=tfidf_transform.transform(message_bow)

In [79]:
from sklearn.naive_bayes import MultinomialNB

In [80]:
spam_detect=MultinomialNB().fit(message_tfidf,messages['Label'])

In [81]:
predict_spam_or_not=spam_detect.predict(message_tfidf)

In [82]:
from sklearn.model_selection import train_test_split

In [84]:
msg_train,msg_test,label_train,label_test=train_test_split(messages['Message'],messages['Label'],test_size=0.3)

In [85]:
from sklearn.pipeline import Pipeline

In [86]:
pipeline_model=Pipeline([
    ('Bow',CountVectorizer(analyzer=text_cleaning)),
    ('tfidf',TfidfTransformer()),
    ('Classifier',MultinomialNB())
])

In [87]:
pipeline_model.fit(msg_train,label_train)

Pipeline(memory=None,
         steps=[('Bow',
                 CountVectorizer(analyzer=<function text_cleaning at 0x000002003E148048>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('Classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [91]:
making_prediction = pipeline_model.predict(msg_test)

In [93]:
print(classification_report(label_test,making_prediction))

              precision    recall  f1-score   support

         ham       0.94      1.00      0.97      1447
        spam       1.00      0.62      0.77       225

    accuracy                           0.95      1672
   macro avg       0.97      0.81      0.87      1672
weighted avg       0.95      0.95      0.94      1672

